In [ ]:
import os
import re
from datetime import date
import matplotlib.dates as mdates
import pandas as pd
import logging
logging.getLogger().setLevel(logging.DEBUG)

import matplotlib.pyplot as plt
import json
import spacy
from copy import deepcopy
from mae_utils import parse_mae

%matplotlib inline

In [ ]:
from spacy_tokenization_init import make_customize_tokenizer_pathology

In [ ]:
from collections import defaultdict, Counter
import spacy
from spacy.tokens import Doc, DocBin, Span
from spacy.matcher import Matcher

Doc.set_extension("id", default=None, force=True)
Doc.set_extension("note_id", default=None, force=True)
Doc.set_extension("note_type", default=None, force=True)
Doc.set_extension("fold", default=None, force=True)
Doc.set_extension("dataset", default=None, force=True)
Doc.set_extension("annotator", default=None, force=True)
Doc.set_extension("entities", default=None, force=True)

In [ ]:
import os
import datetime
def modification_date(filename):
    t = os.path.getmtime(filename)
    return datetime.datetime.fromtimestamp(t)

In [ ]:
def parse_filename(fn, pattern_filename = re.compile("(?P<note_id>\d+)(?:_(?P<annotator>[A-Za-z]+))\.xml")
                  ):
    match = pattern_filename.search(fn)
    if match:
        fn_dict = match.groupdict()
        fn_dict["note_id"] = int(fn_dict["note_id"])
    else:
        fn_dict = {"note_id":None, "annotator":None}
    return fn_dict

In [ ]:
# nlp = spacy.load('en_core_web_sm')
nlp = spacy.load("en_core_web_md")

# move the NER component to the end of the pipeline: remove and then reload from the same source in the new position
nlp.remove_pipe("ner")
nlp.add_pipe("ner", source=spacy.load("en_core_web_sm"))

# add entity ruler
# nlp.add_pipe("entity_ruler", before="ner")
nlp.add_pipe('sentencizer')
nlp = make_customize_tokenizer_pathology()(nlp)
# nlp.add_pipe("morphologizer")

# nlp.initialize()
print(nlp.pipe_names)

In [ ]:
# Add matcher patterns from manually edited json file (note still on 210920_null directory)
with open("data/domain_patterns_dict_manedit.json") as fh:
    domain_patterns_imp = json.load(fh)


####################    added lines  ############################
matcher_lvl1 = Matcher(nlp.vocab)
for domain in domain_patterns_imp:
    matcher_lvl1.add(domain, domain_patterns_imp[domain])
#################################################################

In [ ]:
matcher = Matcher(nlp.vocab)
matcher.add("Social_isolation", 
            [[{
                "LOWER": {"IN":["live", "living", "lives",
                                "stay","stays", "staying", 
                                "reside", "resides", "residing",
                                "housed", "house",]},
            },
            {"LOWER": {"IN":["in", "at"]}, "OP":"?"},
            {"OP":"*"},
            {
                "LOWER": {
                    "IN": [
#                         "in",
                        "together",
                        "with",
                        "w/",
                        "w",
                        "by"
                    ]
                }
            },
            {"LOWER": {"IN":["his", "her", "their", "herself", "himself"]}, "OP":"?"},
            ],
            [{"LOWER": {"IN":["moves", "moved", "move"]}},
             {"LOWER": {"IN":["into", "in", "with"]}}
            ]
            ]
           )

In [ ]:
dates1 = pd.read_csv("data/annotation-dates-batch1.csv")
dates1.datetime = dates1.datetime.map(datetime.datetime.fromisoformat)

In [ ]:
dates1.dtypes
# lambda x: {"Raf": "Rafael",  "Mat": "Matt"}.get(x,x)

In [ ]:
indir  = "data/skippy-2020-02-12-per-type"   # I edited this for my local system

tag_list = []
data = []
for dd in os.scandir(indir):
    print(dd.name)
    for ff in os.scandir(dd):
        if not ff.name.endswith("xml") or not "_" in ff.name:
            continue        
        text, tag_list_ = parse_mae(ff.path, skip_errors=False)
        entry = {"text": text,
                 "entities": tag_list_,
                 "note_type": dd.name
                }
        entry.update(parse_filename(ff.name))
        entry["filename"]  = ff.name
        entry["annotator"] = {"Raf": "Rafael",  
                              "Rafal": "Rafael",
                              "Rafae": "Rafael",
                              "Mtt": "Matt",
                              "Mat": "Matt"}.get(entry["annotator"], entry["annotator"])
        entry["annotator"] = entry["annotator"].title() if entry["annotator"] is not None else None
        entry["datetime"] = modification_date(ff)
        if entry["annotator"] is None:
            continue
        data.append(entry)
        for tag in tag_list_:
            tag["filename"] = ff.name
            tag["note_type"] = dd.name
        tag_list.extend(tag_list_)
len(data)

In [ ]:
note_counts = []
for doc in data:
    if doc["annotator"].title()=="Sam":
        continue
    _c_ = {kk:doc[kk] for kk in ["note_id", "annotator"]}
    _c_["n_entities"] = len(doc["entities"])
    _c_["len"] = len(doc["text"])
    _c_["note_type"] = (doc["note_type"])
    note_counts.append(_c_)

note_counts = (pd.DataFrame(note_counts))
note_counts["annotator"] = note_counts["annotator"].str.title()

note_counts_wide =  note_counts.set_index(["note_id", "annotator"])["n_entities"].unstack()
note_counts_wide.shape

In [ ]:
note_counts_wide.loc[(~note_counts_wide.isnull()).sum(1)==1]

In [ ]:
note_counts_wide.shape

In [ ]:
#

In [ ]:
note_stats = pd.DataFrame({"n_entities": note_counts_wide.mean(1),
 "note_len": note_counts.groupby("note_id")["len"].mean().astype(int),
 "note_type": note_counts.groupby("note_id")["note_type"].first()
             }
            )
note_stats.to_csv("results/note_stats.csv")

In [ ]:
num_of_annotations = (~note_counts_wide.loc[((~note_counts_wide.isnull()).any(1) &
                       (note_counts_wide.sum(1)>0))].isnull()).sum(1).value_counts().sort_index()

In [ ]:
ax = num_of_annotations[::-1].plot(kind="barh")
ax.set_xlim([0, max(num_of_annotations)+70])

for index, row in num_of_annotations[::-1].items():
    ax.text(row+30, 4-index, row, color='black', ha="center", fontsize=14)

ax.set_xlabel("number of documents")
ax.set_ylabel("number of annotators")

In [ ]:
626/1576

In [ ]:
# (~note_counts_wide.isnull()).any(1)

In [ ]:
# (note_counts_wide>0).any(1)

In [ ]:
# pd.merge(
#     note_annotator,
#     note_annotator["note_id"].value_counts()[note_annotator["note_id"].value_counts()<2],
#     right_index=True,
#     left_on = "note_id")

## Spotchecking and correction code

In [ ]:
# for entry in data:
#     for ent in entry["entities"]:
#         if ent["type"] =="Mixed anxiety and depressive disorder":
#             raise ValueError

In [ ]:
# entry

In [ ]:
for entry in data:
    if entry["filename"] == "668198482_Sara.xml":
        break
#     _Matt.xml"
doc = nlp(entry["text"])
print(entry["filename"])

In [ ]:
nn = 0
for tok in doc:
    if tok.text == "personal":
        nn+=1
        if nn > 0:
            break
nn, tok, tok.idx

In [ ]:
ent = doc.char_span(tok.idx, tok.idx+1, label="a", alignment_mode="expand")
longer_ent = doc[ent.start: ent.end+1]
print(longer_ent)
longer_ent.start_char, longer_ent.end_char
print(f'spans="{longer_ent.start_char+1}~{longer_ent.end_char+1}" text="{longer_ent}"')

In [ ]:
for entry in data:
    for ent in entry["entities"]:
        if (ent["label"] == "Housing") & (ent["type"]=="Housed"):
            if ent["text"] == 'lives alone':
                ent["label"] = "Social_isolation"
                ent["type"] = "Lives alone"
            else:
                ent["type"] = "Stably housed"

In [ ]:
with open("./data/sdoh-cocoa-skippy-raw.jsonl", "w") as fh:
    for line in data:
        line = deepcopy(line)
        line["datetime"] = line["datetime"].isoformat()
        fh.write(json.dumps(line)+"\n")

In [ ]:
# json.dumps(line)
# line

In [ ]:
# fn = f"{indir}/Telephone_Encounter/769128562_Sara.xml"
# parse_mae(fn)

In [ ]:
uniq_labels = pd.DataFrame(sum([row["entities"] for row in data], []))[["label", "type"]].drop_duplicates()
uniq_labels.groupby("type")["label"].agg(len)

In [ ]:
# fig, ax = plt.subplots(1)
# ax.scatter(dates1["datetime"], dates1["annotator"])

# ax.xaxis.set_major_locator(mdates.MonthLocator())
# ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))

In [ ]:
# indir  = "data/skippy-2020-02-12"   # I edited this for my local system

# tag_list = []
# data = []

# for ff in os.scandir(indir):
#     if not ff.name.endswith("xml") or not "_" in ff.name:
#         continue        
#     text, tag_list_ = parse_mae(ff.path)
#     entry = {"text": text,
#              "entities": tag_list_,
#              "note_type": dd.name
#             }
#     entry.update(parse_filename(ff.name))
#     entry["filename"]  = ff.name
#     entry["annotator"] = {"Raf": "Rafael",  
#                           "Rafal": "Rafael",
#                           "Rafae": "Rafael",
#                           "Mtt": "Matt",
#                           "Mat": "Matt"}.get(entry["annotator"], entry["annotator"])
#     entry["annotator"] = entry["annotator"].title() if entry["annotator"] is not None else None
#     entry["datetime"] = modification_date(ff)
#     if entry["annotator"] is None:
#         continue
#     data.append(entry)
#     for tag in tag_list_:
#         tag["filename"] = ff.name
# #         tag["note_type"] = dd.name
#     tag_list.extend(tag_list_)
# len(data)

In [ ]:
# pd.Series([entry["datetime"] for entry in data]).value_counts().plot()
# plt.ylim([0, 18])

In [ ]:
fig, ax = plt.subplots(1,)

for annotator, vv in (
    pd.DataFrame([{"datetime":entry["datetime"], "annotator": entry["annotator"]} for entry in data])
    .value_counts(["annotator", "datetime"])
    .groupby("annotator")

):
    ax.plot(vv[annotator], marker=".", ls="", label=annotator)
    
ax.set_ylim([0, 6])
ax.set_xlim([datetime.datetime.fromisoformat("2021-02-01"),
          datetime.datetime.fromisoformat("2021-07-15")])

xticks = sum([[datetime.datetime.fromisoformat(f"2021-{mm:02d}-01"),
 datetime.datetime.fromisoformat(f"2021-{mm:02d}-15")] for mm in range(4,8)], [])

xticks = [datetime.datetime.fromisoformat(f"2021-{mm:02d}-01") for mm in range(1,8)]

# plt.gca().xaxis.set_major_locator(mdates.WeekdayLocator())

ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
# plt.xticks(rotation=45)

plt.legend()

In [ ]:
# pd.DataFrame([{"filename": entry["filename"], "datetime":entry["datetime"], "annotator": entry["annotator"]} 
#               for entry in data]).to_csv("data/annotation-dates-batch1.csv", index=False)

In [ ]:
with open("./data/sdoh-cocoa-skippy-all-2022-01-07.jsonl", "w") as fh:
    for entry in data:
        entry["datetime"] = entry["datetime"].strftime("%Y-%m-%d")
        fh.write(json.dumps(entry) + "\n")

In [ ]:
def get_meta(entry):
    output = {kk:vv for kk, vv in entry.items() if kk not in {"text", "entities"}}
    output.update({"n_entities": len(entry["entities"])})
    return output

metadata = pd.DataFrame([get_meta(entry)
                         for entry in data]).set_index("note_id")
metadata.shape

In [ ]:
metadata.groupby(level="note_id").count()["filename"].shape

In [ ]:
# metadata.groupby("note_type").plot()

In [ ]:
metadata.groupby("annotator").count()

In [ ]:
metadata.index.value_counts().value_counts()

In [ ]:
metadata.groupby(level=0)["n_entities"].max().value_counts()

In [ ]:
metadata.groupby("note_type").sum()

In [ ]:
metadata.groupby(["note_type", "annotator"]).sum()["n_entities"].unstack()

In [ ]:
metadata.index.unique().shape

In [ ]:
metadata.reset_index().groupby(["note_type", "note_id"])["n_entities"].mean()[metadata.reset_index().groupby(["note_type", "note_id"])["n_entities"].mean()>0
                                                                             ].groupby("note_type").sum()


In [ ]:
metadata[metadata.n_entities>0].index.unique().shape

In [ ]:
note_ids = pd.concat({"counts": metadata.index.value_counts(),
                      "n_entities_max": metadata.groupby(level=0)["n_entities"].max()},1)

In [ ]:
# metadata

In [ ]:
note_ids["n_entities_bucket"] = pd.cut(note_ids["n_entities_max"], [-1,0,1,5,10, 100])

In [ ]:
note_ids["buckets"] = note_ids.apply(lambda row: (row["counts"], row["n_entities_bucket"].left, row["n_entities_bucket"].right), 1)
note_ids["buckets"].value_counts()

In [ ]:
note_ids.loc[note_ids["buckets"].map(lambda row: row[0]==5), "buckets"].value_counts()

In [ ]:
note_ids["buckets_large"] = note_ids["buckets"].map(lambda row: {(5, 0, 1):(4, 0, 1),
                                     (5, 10, 100): (None, 10, 100),
                                     (4, 10, 100): (None, 10, 100),
                                     (3, 10, 100): (None, 10, 100),
#                                      (2, 10, 100): (None, 10, 100),
                                    }.get(row, row)).value_counts()

In [ ]:
# note_ids.loc[note_ids["buckets"].map(lambda row: row[0]==5), "buckets"].value_counts()

In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold

In [ ]:
skf = StratifiedKFold(6)
note_id_set = pd.Series({ii:note_ids.index[vv] for ii, (tt,vv)
                         in enumerate(skf.split(note_ids.index, note_ids["buckets_large"].astype(str)))}).explode()
note_id_set.index.name =  "fold"
note_id_set.name =  "note_id"
note_id_set = note_id_set.reset_index()
note_id_set["set"] = note_id_set["fold"].map(lambda x: {4:"val", 5:"test"}.get(x,"train"))
note_id_set = note_id_set.set_index("note_id")

In [ ]:
note_id_set["set"].value_counts()

In [ ]:
if "set" not in metadata.columns:
    metadata = pd.merge(metadata, note_id_set, left_index=True, right_index=True)

In [ ]:
metadata.index.unique().shape

In [ ]:
metadata[metadata.n_entities>0].index.unique().shape

In [ ]:
metadata.to_csv("data/skippy-2020-02-12-data-splits-2022-01-11.csv")

In [ ]:
import span_utils
import overlap

import importlib
span_utils = importlib.reload(span_utils)
overlap = importlib.reload(overlap)

In [ ]:
def trim_whitespaces(spans):
    for sp in spans:
        if (doc[sp.end].is_space):
            for ii in range(sp.end, sp.start-1, -1):
                if not (doc[ii].is_space):
                    sp.end = ii
                    break
        if (doc[sp.start].is_space):
            for ii in range( sp.start, sp.end):
                if not (doc[ii].is_space):
                    sp.start = ii
                    break
    return spans

In [ ]:
def trim_whitespaces_dict(spans, text,
                         chars = {" ", "\n", "\t", ",", ":", ".", ";", "-"}):
    for sp in spans:
        if (text[sp["end"]] in chars):
            for ii in range(sp["end"], sp["start"]-1, -1):
                if not (text[ii] in chars):
                    sp["end"] = ii+1
                    break
        if (text[sp["start"]] in chars):
            for ii in range( sp["start"], sp["end"]):
                if not (text[ii] in chars):
                    sp["start"] = ii
                    break
    return spans

In [ ]:
from collections import defaultdict

def split_overlapping_spans(snippet, labels = []):
    matches = matcher_lvl1(snippet, with_alignments = True)
    spans = defaultdict(lambda : [])
    for hash_, start, end, algn in matches:
        label = matcher_lvl1.vocab.strings[hash_]
        span = snippet[start: end]
        span.label_ = label
        spans[label].append((span, algn[-1] - algn[0]))
        
    new_spans = []
    for kk, vv in spans.items():
        if len(labels) and kk not in labels:
            continue
        vv = spacy.util.filter_spans([x[0] for x in vv])
#         vv = sorted(enumerate(vv), key = lambda x: x[1][1])[-1][1][0]
        new_spans.extend(vv)
    return new_spans

In [ ]:
# snippet = nlp("Lives at home with")
# snippet = nlp("Sister and half brother with depression")
# spans_disamb = split_overlapping_spans(snippet, labels =  ["Social_isolation", "Depression"])
# # spans_disamb = sorted(spans_disamb, key = lambda x : ( x.end))
# spans_disamb

In [ ]:
try:
    Span.set_extension("type", default=None)
except Exception as ee:
    print(ee)

In [ ]:
from collections import Counter

def recover_lives_with(sp, margin = 20, matcher=matcher):
    doc = sp.doc
    snippet = doc[sp.start-margin: sp.end+margin]
    matches = matcher(snippet, with_alignments=True)
#     return matches
    if len(matches)>0:
        index = max(enumerate([alignment[-1] for _, start, end, alignment in matches]), 
                    key=lambda x: x[1])[0]
        _, start, end, alignment = matches[index]
#         logging.debug(index)
#         max_len_ = 0
#         longest_span = None
#         for _, start, end in matches:
# #         match = matches[0]
        longest_span = doc[sp.start-margin + start: sp.start-margin + end]
#         if len(temp_span)> max_len_:
#             longest_span = temp_span
#         max_len_ = len(temp_span)
        longest_span.label = sp.label
        for attr in dir(sp._):
            if attr not in {'get', 'has', 'set'}:
                longest_span._.set(attr, getattr(sp._, attr))

        return longest_span
    else:
        logging.warning(f"no matches in: '{snippet}'")
#         raise Exception
        return sp

In [ ]:
# snippet = nlp('''for laboratory services at UCSF Center for Reproductive Health.    
# Lives in San Francisco with mother, brother. Lives in Teotihuacan on the weekends with her boyfriend.   
# Exercise: 60-90 min on 3 days/''')[0:]
# matcher(snippet, with_alignments=True)

In [ ]:
# recover_lives_with(snippet)

In [ ]:
def recover_dict_lives_with(text, start, end, margin = 20, matcher=matcher):
    doc = sp.doc
    snippet = text[sp.start-margin: sp.end+margin]
    matches = matcher(snippet)
    if len(matches)>0:
        index = max(enumerate([alignment[-1] for _, start, end, alignment in matches]), key=lambda x: x[1])[0]
        _, start, end, alignment = matches[index]
        print(index)
#         max_len_ = 0
#         longest_span = None
#         for _, start, end in matches:
# #         match = matches[0]
        temp_span = doc[sp.start-margin + start: sp.start-margin + end]
        if len(temp_span)> max_len_:
            longest_span = temp_span
        max_len_ = len(temp_span)
        longest_span.label = sp.label
        for attr in dir(sp._):
            if attr not in {'get', 'has', 'set'}:
                longest_span._.set(attr, getattr(sp._, attr))

        return longest_span
    else:
        logging.warning(f"no matches in: '{snippet}'")
#         raise Exception
        return sp

In [ ]:
# %%cython
# from libcpp.vector cimport vector
# def overlap(int x1, int x2, int y1, int y2):
# #     if x1 < y2 or y1<x2:
#     length = max(0, max(x2, y2) - min(x1,y1))
#     if (length)>0:
#         return max(0, (min(x2, y2) - max(x1,y1)) / length)
#     else:
#         return 0

def get_priority(x, y, priority_dict):
    return x if priority_dict[x] <= priority_dict[y] else y
    
def clean_overlaps_disjoin(spans, priority_dict = {}):
    sp_prev = None
#     cdef vector[int] remove_list
    remove_list  = []
    for ii, sp in enumerate(spans):

        if sp_prev is None:
            sp_prev = sp
            continue

        ov = overlap.overlap(sp["start"], sp["end"], sp_prev["start"], sp_prev["end"])
#         print(ov, sp["start"], sp["end"], sp_prev["start"], sp_prev["end"], sp["text"])
        if ov > 0.0:
            if abs(sp["end"] - sp_prev["end"]) > abs(sp["start"] - sp_prev["start"]): 
                if sp["end"] > sp_prev["end"]:
                    sp["start"] = sp_prev["end"]
                else:
                    sp_prev["start"] = sp["end"]
#                 remove_list.push_back(ii-1)
#                 remove_list.append(ii-1)
            elif abs(sp["end"] - sp_prev["end"]) > abs(sp["start"] - sp_prev["start"]):
                if sp["start"] < sp_prev["start"]:
                    sp["end"] = sp_prev["start"]
                else:
                    sp_prev["end"] = sp["start"]
#                 remove_list.push_back(ii)
#                 remove_list.append(ii)
            else: 
                logging.debug(f"ties:{sp}\n{sp_prev}")
                if len(priority_dict)>0:
                    if priority_dict.get(sp["label"], 1e20) <= priority_dict.get(sp_prev["label"], 1e20):
#                         sp_prev["label"] = None
#                         remove_list.push_back(ii-1)
                        remove_list.append(ii-1)
                    else:
#                         sp["label"] = None
                        remove_list.append(ii)
        sp_prev = sp
#     print("remove_list", remove_list)
    for ii in reversed(remove_list):
        spans.pop(ii)
    return spans

In [ ]:
def clean_overlaps_disjoin_spacy(spans, priority_dict = {},
                                 label_fn = lambda x: x.split(': ')[0]):
    doc = spans[0].doc
    sp_prev = None
#     cdef vector[int] remove_list
    remove_list  = []
    new_spans = []
    prev_ranges = []
    
    for ii, sp in enumerate(spans):
        
#         if ("daughter" in sp.text) and ("daughter" != sp.text.lower()):
#             raise Exception
            
        if sp_prev is None:
            sp_prev = sp
            new_spans.append(sp)
            prev_ranges.append((sp.start, sp.end))
            continue

        ov = overlap.overlap(sp.start, sp.end, sp_prev.start, sp_prev.end)
        #logging.debug(f"{sp_prev.start} -- {sp_prev.end}: {sp.start} -- {sp.end}, overlap = {ov}")
        #logging.debug(f"this one:{abs(sp.end - sp.start)}\nprevios:{abs(sp_prev.end - sp_prev.start)}")
#         print(ov, sp.start, sp.end, sp_prev.start, sp_prev.end, sp["text"])
        if ov > 0.0:
            if sp_prev.label == sp.label:
                this_len = sp.end - sp.start
                prev_len = sp_prev.end - sp_prev.start
                if this_len > prev_len:
                    logging.debug(f"keeping the longer: {sp} ({sp.label_})")
                    remove_list.append(ii-1)
                else:
                    logging.debug(f"keeping the longer: {sp_prev} ({sp_prev.label_})")
                    remove_list.append(ii)
                if sp_prev is not None:
                    new_spans[-1] = sp_prev
                new_spans.append(sp)
                sp_prev = sp
                prev_ranges.append((sp.start, sp.end))
                continue
            # tail overhang is longer then head overhang
            relevant_starts = [s for s, e in prev_ranges if e > sp.start]
            logging.debug(f"relevant_starts: {relevant_starts}")
            tail_overhang = abs(sp.end - sp_prev.end)
            head_overhang = abs(sp.start - min(relevant_starts))
            flag_tail = (sp.label_ == "Social_isolation: Lives with") or \
                (sp_prev.label_ == "Social_isolation: Lives with") and  sp.end != sp_prev.end
            
            if tail_overhang > head_overhang or flag_tail:
                logging.debug(f"tail overhang ({tail_overhang}) is longer then head overhang({head_overhang})")
                logging.debug(f"fixing tails")
                if sp.end > sp_prev.end:
#                     sp.start = sp_prev.end
                    label = sp.label_
                    sp = doc[sp_prev.end:sp.end]
                    sp.label_ =  label
                    logging.debug(f"({sp_prev.label_}) {sp_prev} >({sp_prev.end}){sp} ({label})")
                elif sp.end < sp_prev.end:
#                     sp_prev.start = sp.end
                    label = sp_prev.label_
                    sp_prev = doc[sp.end:sp_prev.end]
                    sp_prev.label_ = label
                    logging.debug(f"({sp.label_}) {sp} >({sp.end}) {sp_prev} ({label})")
#                 remove_list.push_back(ii-1)
#                 remove_list.append(ii-1)
                else:
                    raise ValueError(f"this.end == prev.end: ({sp}){sp.end}== ({sp_prev}){sp_prev.end}")
            elif abs(sp.end - sp.start) != abs(sp_prev.end - sp_prev.start): # looser criterion
#             elif abs(sp.end - sp_prev.end) > abs(sp.start - sp_prev.start):
                if tail_overhang < head_overhang:
                    logging.debug(f"tail overhang ({tail_overhang}) is shorter then head overhang({head_overhang})")
                else:
                    logging.debug(f"tail overhang ({tail_overhang}) is equal to head overhang({head_overhang})")
                logging.debug(f"fixing heads")
                if sp.start < sp_prev.start:
#                     sp.end = sp_prev.start
                    label = sp.label_
                    sp = doc[sp.start:sp_prev.start]
                    sp.label_ =  label
                    logging.debug(f"({sp.label_}) {sp}< {sp_prev} ({sp_prev.label_})")
                else:
#                     sp_prev.end = sp.start
                    label = sp_prev.label_
                    sp_prev = doc[sp_prev.start:sp.start]
                    sp_prev.label_ = label
                    logging.debug(f"({sp_prev.label_}) {sp_prev}< {sp} ({sp.label_})")
#                 remove_list.push_back(ii)
#                 remove_list.append(ii)
            else: 
#                 logging.debug(f"ties:({sp}) {sp.label_}:{sp._.type} -- {sp_prev.label_}:{sp_prev._.type}")
                if len(priority_dict)>0:
                    rank_this = priority_dict.get(label_fn(sp.label_), 1e20)
                    rank_prev = priority_dict.get(label_fn(sp_prev.label_), 1e20)
                    if rank_this <= rank_prev:
#                         sp_prev["label"] = None
#                         remove_list.push_back(ii-1)
                        logging.debug(f"ties:({sp}) keep {sp.label_} ({rank_this}) "
                                      f"-- remove {sp_prev.label_} ({rank_prev})")
#                         logging.debug(f"ties:({sp}) keep {sp.label_}:{sp._.type} ({rank_this}) "
#                                       f"-- remove {sp_prev.label_}:{sp_prev._.type} ({rank_prev})")
                        remove_list.append(ii-1)
                    else:
#                         sp["label"] = None
                        logging.debug(f"ties:({sp}) keep {sp_prev.label_} ({rank_prev}) "
                                      f"-- remove {sp.label_} ({rank_this})")
                        remove_list.append(ii)
                else:
                    logging.debug(f"ties:({sp}) keep both: {sp.label_}, {sp_prev.label_}")
#         print("ii", ii, new_spans)
        if sp_prev is not None:
            new_spans[-1] = sp_prev
        new_spans.append(sp)
        sp_prev = sp
        prev_ranges.append((sp.start, sp.end))
        
#     print("remove_list", remove_list)
    for ii in reversed(remove_list):
        new_spans.pop(ii)
    return new_spans

In [ ]:
# clean_overlaps_disjoin_spacy((spans), 
# #                                                                  key = lambda x : (x["start"], x["end"])),
#                                              priority_dict={"Marital_or_partnership_status":0,
#                                                "Social_isolation":1,
#                                                "Housing":2})

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
def _collect_overlapping_groups(spans):
    doc = spans[0].doc
    sp_prev = None
#     cdef vector[int] remove_list
    groups = defaultdict(lambda : set())
    gr = 1
    new_spans = []
    for ii, sp in enumerate(spans):

        if sp_prev is None:
            sp_prev = sp
            new_spans.append(sp)
            continue

        ov = overlap.overlap(sp.start, sp.end, sp_prev.start, sp_prev.end)
        if ov > 0.0:
            groups[gr].add(sp_prev)
            groups[gr].add(sp)
        else:
            gr+=1
            groups[gr].add(sp)
        sp_prev = sp
    return groups

def _split_group_with_matcher(spans_):
    """input: a group of overlapping labelled spans (1, 2, or more)
    output: disjoint labelled spans separated using pattern matching
    """
    if len(spans_)==1:
        return spans_
    start_ = min(spans_, key=lambda x: x.start).start
    end_ = max(spans_, key=lambda x: x.end).end
    labels = set([sp.label_ for sp in spans_])
    sp_ = spans_.pop()
#     if hasattr(sp_._, "type"):
#         types = {}
#         for sp in spans_:
#             types[sp.label_] = sp._.type
    snippet = sp_.doc[start_:end_]
    spans_ = split_overlapping_spans(snippet, labels=labels)

#     if hasattr(sp_._, "type"):
#         for sp_ in spans_:
#             sp_._.type = types[sp_.label_]
    return spans_

def split_overlaps_matcher_spacy(spans, priority_dict = {}):
    groups = _collect_overlapping_groups(spans)
        
    result = []
    for gr, spans_ in groups.items():
        spans_ = _split_group_with_matcher(spans_)
        result.extend(spans_)
            
    return result

In [ ]:
# ii = 0
# for entry in data:
#     for ent in entry["entities"]:
#         if ent["label"] == "Housing":
#             print(ent["type"], ent["text"], sep=" | ")
#             if ent["type"]=="Marginally housed":# and ent["text"] == "apartment":
#                 ii+=1
#                 if ii>2:
#                     raise Exception

In [ ]:
# entry

In [ ]:
def process_note(entry):
    doc = nlp(entry["text"])
    doc._.note_id = entry["note_id"]
    doc._.id = str(entry["note_id"]) + "_" + entry["annotator"]
    doc._.dataset = note_id_set.loc[entry["note_id"], "set"]
    doc._.fold = int(note_id_set.loc[entry["note_id"], "fold"])
    doc._.annotator = entry["annotator"]
    doc._.note_type = entry["note_type"]
    
    if len(entry["entities"])==0:
        return None
    entities = trim_whitespaces_dict(entry["entities"], doc.text)
    entities = sorted(entities, key = lambda x : (x["start"], x["end"]))
    spans = [doc.char_span(ent["start"], ent["end"], label=ent["label"] + ": " + ent["type"],
                           vector=None, alignment_mode="expand")
                  for ent in entities
            ]
    new_spans = []
    for sp, ent in zip(spans, entities):

        if sp.label_ == "Marital_or_partnership_status: Partner relationship problem" and \
            sp.text in {"husband", "wife", "partner", "spouse", "child", "son", "daughter"}:
            raise Exception

        if any((x in sp.text.lower() for x in ["home", "housed", "lives at home", "apartment", "residence",
                               "condo", "house", "lives in home", "one bedroom apartment","relocated",
                              "housing"])) and \
                sp.label_ == "Housing: NA":
            sp.label_ = "Housing: Housed"
        elif any((x in sp.text.lower() for x in ['boyfriend', 'girlfriend', 'gf', 'bf', "rdp",])) and \
                sp.label_ == "Marital_or_partnership_status: Single":
#             sp.label_ = "Social_isolation: Has social support"
            sp.label_ == "Marital_or_partnership_status: Partner in a relationship"
#             if sp is not None:
#                 sp._.type = ent["type"]
        elif sp.label_ == "Anxiety: Level of anxiety": # only Sara used this label (3 times)
            sp.label_ = "Anxiety: Anxiety"
        elif sp.label_ == "Social_isolation: Support system case management": # only 1 label in this class
            sp.label_ = "Social_isolation: NA"
        elif (sp.label_ == "Social_isolation: Lives with") and (
                ("with" not in sp.text.lower()) and 
                ("w/" not in sp.text.lower()) and 
                ("into" not in sp.text.lower())
                ):
            logging.debug(f"original lives_with:'{sp}'--{sp.label_}")
            sp = recover_lives_with(sp)
            logging.debug(f"recovered lives_with:'{sp}'--{sp.label_}")
#                 break
        if (sp.label_ != "Social_isolation: Lives with") and ("Lives with" in sp.text.lower()):
            raise ValueError
#                 break
        new_spans.append(sp)
    new_spans = sorted(new_spans, key = lambda x : ( x.end))
    new_spans = clean_overlaps_disjoin_spacy((new_spans), 
#                                                                  key = lambda x : (x["start"], x["end"])),
                                         priority_dict={"Marital_or_partnership_status":0,
                                           "Social_isolation":1,
                                           "Housing":2})

    for sp in new_spans:
        if sp.text.lower()=="italian":
            raise Exception
        if "lives at" == sp.text.lower() and sp.label_ == "Social_isolation: Lives with":
            raise Exception
#             if sp.label_ == "Depression: PHQ-9":
#                 raise Exception
        if sp.label_ == "Social_isolation: Housing: NA":
            raise Exception
        if sp.label_ == "Housing: NA" and sp.text not in  {"homes", "rent", "living situation"}:
            raise Exception
        if sp.label_ == "Marital_or_partnership_status: Partner relationship problem" and \
            sp.text in {"husband", "wife", "partner", "spouse", "child", "son", "daughter"}:
            raise Exception
        if sp.label_ == "Social_isolation: Lives alone" and sp.text == "Lives at":
            raise Exception

        if any((x in sp.text.lower() for x in ['boyfriend', 'girlfriend', 'gf', 'bf'])) and \
                sp.label_ == "Marital_or_partnership_status: Single":
            raise Exception
#         new_spans = sorted(new_spans, key = lambda x : (x.start, x.end))
    filtered_spans = spacy.util.filter_spans([sp for sp in new_spans if sp is not None]) 
    if len(filtered_spans) < len(new_spans):
        removed_ents = list(set(new_spans) - set(filtered_spans))
        for ent in removed_ents:
            logging.warning(f"{ii}\tremoved:\t{ent}\t{ent.label_}")
        raise Exception

    doc.set_ents((filtered_spans))
    return doc

In [ ]:
# span_utils.get_ent_df(doc.ents)

In [ ]:
# span = doc[1907:1912]
# print(span.text.strip(), "\n", span.start_char+1, span.end_char+1)

In [ ]:
# span = doc[1943:1946]
# print(span.text.strip(), "\n", span.start_char+1, "~", span.end_char+1, sep="")

In [ ]:
remapping = {"Marital_or_partnership_status: Common law partnership": "Marital_or_partnership_status: Partner",
"Marital_or_partnership_status: Divorce": "Marital_or_partnership_status: Divorced",
 "Anxiety: Anxiety: mood/finding": "Anxiety: Signs and symptoms of anxiety",
 "Depression: Depressed mood": "Depression: Symptoms of depression"
}

In [ ]:
docs = []
break_ = False
for ii, entry in enumerate(data):
    if ii < 0:
        continue
    if ii%50 == 0:
        print(ii, end="\t")
    if note_ids.loc[entry["note_id"], "n_entities_max"] == 0:
        continue
    doc = nlp(entry["text"])
    doc._.note_id = entry["note_id"]
    doc._.id = str(entry["note_id"]) + "_" + entry["annotator"]
    doc._.dataset = note_id_set.loc[entry["note_id"], "set"]
    doc._.fold = int(note_id_set.loc[entry["note_id"], "fold"])
    doc._.annotator = entry["annotator"]
    doc._.note_type = entry["note_type"]
    try:
        if len(entry["entities"])==0:
            continue
        entities = trim_whitespaces_dict(entry["entities"], doc.text)
        entities = sorted(entities, key = lambda x : (x["start"], x["end"]))
        spans = [doc.char_span(ent["start"], ent["end"], label=ent["label"] + ": " + ent["type"],
                               vector=None, alignment_mode="expand")
                      for ent in entities
                ]
        new_spans = []
        for sp, ent in zip(spans, entities):
            
            if sp.label_ == "Marital_or_partnership_status: Partner relationship problem" and \
                sp.text in {"husband", "wife", "partner", "spouse", "child", "son", "daughter"}:
                raise Exception
                
            if any((x in sp.text.lower() for x in ["home", "housed", "lives at home", "apartment", "residence",
                                   "condo", "house", "lives in home", "one bedroom apartment","relocated",
                                  "housing"])) and \
                    sp.label_ == "Housing: NA":
                sp.label_ = "Housing: Housed"
                
            if ("section 8" in sp.text.lower()) or \
                ("public housing" in sp.text.lower()):
                sp.label_ = "Housing: Subsidized housing"
                
            if ("St Joseph's apartment" in sp.text) or ("group home" in sp.text.lower()):
                sp.label_ = "Housing: lives in facility"
                
            if any((x in sp.text.lower() for x in ['boyfriend', 'girlfriend', 'gf', 'bf', "regdompart", "rdp"])):# and \
                sp.label_ = "Marital_or_partnership_status: Partner"
#             if sp.label_ == "Marital_or_partnership_status: Common law partnership":
#                 sp.label_ = "Marital_or_partnership_status: Partner"

            if sp.label_.startswith("Marital_or_partnership_status:") and "widow" in sp.text.lower():
                sp.label_ = "Marital_or_partnership_status: Widowed"

            if sp.label_.startswith("Marital_or_partnership_status:") and "divorce" in sp.text.lower():
                sp.label_ = "Marital_or_partnership_status: Divorced"
                
            if sp.label_ == "Depression: NA" and sp.text.lower() == "depression":
                sp.label_ == "Depression: Symptoms of depression"
        
            if sp.label_ == "Anxiety: NA" and sp.text.lower() == "anxiety":
                sp.label_ == "Anxiety: Signs and symptoms of anxiety"
                
            if (sp.label_ == "Social_isolation: Lives with") and (
                    ("with" not in sp.text.lower()) and 
                    ("w/" not in sp.text.lower()) and 
                    ("into" not in sp.text.lower())
                    ):
                logging.debug(f"original lives_with:'{sp}'--{sp.label_}")
                sp = recover_lives_with(sp)
                logging.debug(f"recovered lives_with:'{sp}'--{sp.label_}")
#                 break
            if (sp.label_ != "Social_isolation: Lives with") and ("Lives with" in sp.text.lower()):
                raise ValueError
#                 break
            # ======= Remap labels
            sp.label_ = remapping.get(sp.label_, sp.label_)
        
            new_spans.append(sp)
        new_spans = sorted(new_spans, key = lambda x : ( x.end))
        new_spans = clean_overlaps_disjoin_spacy((new_spans), 
#                                                                  key = lambda x : (x["start"], x["end"])),
                                             priority_dict={"Marital_or_partnership_status":0,
                                               "Social_isolation":1,
                                               "Housing":2})
#         doc._.entities = new_spans
        for sp in new_spans:
            if sp.text.lower()=="italian":
                raise Exception
            if "lives at" == sp.text.lower() and sp.label_ == "Social_isolation: Lives with":
                raise Exception
#             if sp.label_ == "Depression: PHQ-9":
#                 raise Exception
            if sp.label_ == "Social_isolation: Housing: NA":
                raise Exception
            if sp.label_ == "Housing: NA" and sp.text not in  {"homes", "rent", "living situation"}:
                raise Exception
            if sp.label_ == "Marital_or_partnership_status: Partner relationship problem" and \
                sp.text in {"husband", "wife", "partner", "spouse", "child", "son", "daughter"}:
                raise Exception
            if sp.label_ == "Social_isolation: Lives alone" and sp.text == "Lives at":
                raise Exception
                
            if any((x in sp.text.lower() for x in ['boyfriend', 'girlfriend', 'gf', 'bf'])) and \
                    sp.label_ == "Marital_or_partnership_status: Single":
                raise Exception
#         new_spans = sorted(new_spans, key = lambda x : (x.start, x.end))
        filtered_spans = spacy.util.filter_spans([sp for sp in new_spans 
                                                  if (sp is not None) and (sp.label_ not in {"Misc: NA", })]) 
        if len(filtered_spans) < len(new_spans):
            removed_ents = list(set(new_spans) - set(filtered_spans))
            for ent in removed_ents:
                logging.warning(f"{ii}\tremoved:\t{ent}\t{ent.label_}")
#             raise Exception
            
        doc.set_ents((filtered_spans))
#         break
    except Exception as ee:
        print(ii)
        raise ee
    docs.append(doc)
print()

In [ ]:
# tok2vec = dict(nlp.pipeline)["tok2vec"]

In [ ]:
# doc_ = deepcopy(doc)
# scores = tok2vec.predict(docs)

if sp.label_.startswith("Marital_or_partnership_status:") and "widow" in sp.text.lower():
    sp.label_ = "Marital_or_partnership_status: Widowed"

if sp.label_.startswith("Marital_or_partnership_status:") and "divorce" in sp.text.lower():
    sp.label_ = "Marital_or_partnership_status: Divorced"

In [ ]:
data_after_spacy = []
break_ = False
for doc in docs:
    dict_ = {kk:vv for kk,vv in doc.to_json().items() if kk in ['text', 'ents']}
    for sp, ent in zip(doc.ents, dict_["ents"]):
        ent["type"] = sp._.type
        ent["text"] = sp.text
        if sp._.type=="Has social support" and sp.label_ != "Social_isolation":
            print("!!!")
            break_ = True
            break
    dict_.update({kk:getattr(doc._, kk) for kk in dir(doc._) if isinstance(getattr(doc._, kk), (str, int))})
    data_after_spacy.append(dict_)
    if break_:
        break
len(data_after_spacy)

In [ ]:
relabel_dict = {"Insurance_status: Able to access healthcare services": "Insurance_status: NA",
                "Insurance_status: Education about insurance coverage": "Insurance_status: NA",
                "Housing: Housed":"Housing: Stably housed",
                # "Anxiety: Anxiety: mood/finding" : "Anxiety: Anxiety",
                "Marital_or_partnership_status: Single": "Marital_or_partnership_status: Single person",
                "Social_isolation: Support system case management" : "Social_isolation: NA",
                "Food: Food Insecurity": "Food: Provision of food",
                "Financial_strain: Low Income": "Financial_strain: Financial problem",
                "Financial_strain: Unable to afford visit copayment": "Financial_strain: Financial problem",
                }

In [ ]:
for doc in docs:
    doc_ents = []
    for ent in doc.ents:
        if ent.label_ in relabel_dict:
            ent.label_ = relabel_dict[ent.label_]
        elif "Depression: Mixed anxiety and depressive disorder" ==  ent.label_:
            break
        doc_ents.append(ent)
        doc.set_ents(doc_ents)
#             print(ent.label_)
#     break

In [ ]:
from span_utils import get_ent_df

In [ ]:
def _tok_to_dict(ent, snippet=None):
    result = {"text": ent.text,
              "start": ent.start_char, "end": ent.end_char,
              "start_tok": ent.start, "end_tok": ent.end,
              "label": ent.label_}

    for ext in dir(ent._):
        if ext not in {"has", "set", "get"}:
            val = getattr(ent._, ext)
            if val is not None:
                result[ext] = val

    if hasattr(ent._, "type"):
        result["type"] = ent._.type

    if snippet:
        doc = ent.doc
        result["snippet"] = doc[max(0, ent.start-snippet):min(ent.end+snippet, len(doc))].text
    return result

def get_ent_dict(ents, snippet = None):
    return [_tok_to_dict(ent, snippet=snippet) for ent in ents]


def get_ent_df(ents, snippet = None):
    ents = get_ent_dict(ents, snippet = snippet)
    return pd.DataFrame(ents)

In [ ]:
df_ents = []
for doc in docs:
    df_ = get_ent_df(doc.ents, snippet=10)
    df_["id"] = doc._.id
    df_ents.append(df_)

df_ents = pd.concat(df_ents)
df_ents.shape

In [ ]:
!open results/label_counts_lvl2_normalized.csv

In [ ]:
df_ents["label"].value_counts().sort_index().to_frame().to_csv("results/label_counts_lvl2_normalized.csv")

In [ ]:
df_ents["label"].value_counts()[df_ents["label"].value_counts()<30]

In [ ]:
# df_ents[df_ents["label"] == "Housing: lives in facility"]

In [ ]:
df_ents[df_ents["label"] == "Anxiety: NA"]["text"].value_counts()

In [ ]:
df_ents[(df_ents["label"] == "Anxiety: NA") &
       (df_ents["text"].str.lower() == "anxiety")]["snippet"].str.strip().str.replace("\n"," ").map(print)

In [ ]:
len(set([doc._.note_id for doc in docs]))

In [ ]:
docbins = defaultdict(lambda: DocBin(store_user_data=True))
for doc in docs:
    docbins[doc._.dataset].add(doc)

for set_, docbin_ in docbins.items():
    docbin_.to_disk(f"./data/spacy-level-2/sdoh-cocoa-skippy-no-empty-2022-01-11-{set_}.spacy")

In [ ]:
for fold in range(6):
    docbins = defaultdict(lambda: DocBin(store_user_data=True))
    for doc in docs:
        if doc._.fold == fold:
#             print(fold, "val")
            docbins["val"].add(doc)
        elif (doc._.fold  + 1 ) % 6 == fold:
#             print(fold, "test")
            docbins["test"].add(doc)
        else:
            docbins["train"].add(doc)
            
    for set_, docbin_ in docbins.items():
        docbin_.to_disk(f"./data/spacy-level-2/sdoh-cocoa-skippy-no-empty-2022-01-11-fold-{fold}-{set_}.spacy")
#     docbins = defaultdict(lambda: DocBin(store_user_data=True))
#     for doc in docs:
#         docbins[doc._.dataset].add(doc)

#     for set_, docbin_ in docbins.items():
#         docbin_.to_disk(f"./data/sdoh-cocoa-skippy-no-empty-{set_}.spacy")
    

In [ ]:
!ls ./data/spacy-level-2/sdoh-cocoa-skippy-no-empty-2022-01-11-fold-*

In [ ]:
with open("./data/sdoh-cocoa-skippy-no-empty-2022-01-11.jsonl", "w") as fh:
    for doc in docs:
        dict_ = {kk:vv for kk,vv in doc.to_json().items() if kk in ['text', 'ents']}
        for sp, ent in zip(doc.ents, dict_["ents"]):
            ent["type"] = sp._.type
            ent["text"] = sp.text
        dict_.update({kk:getattr(doc._, kk) for kk in dir(doc._) if isinstance(getattr(doc._, kk), (str, int))})
        fh.write(json.dumps(dict_) + "\n")

In [ ]:
# {kk:getattr(doc._, kk) for kk in dir(doc._) if isinstance(getattr(doc._, kk), (str, int))}

In [ ]:
type(doc._.fold)

In [ ]:
# !head -n1 "./data/sdoh-cocoa-skippy-no-empty.jsonl"

In [ ]:
entities = []
for doc in docs:
    entities.extend([{"label":ent.label_, "text":str(ent), "doc_id": doc._.id} for ent in doc.ents])
df_entities = pd.DataFrame(entities)

In [ ]:
# df_entities.label.value_counts()

In [ ]:
df_entities.groupby("label")["text"].value_counts()[:20]#["pain_and_disability"]

In [ ]:
# import matplotlib.pyplot as plt

# %matplotlib inline

# plt.hist([len(doc.ents) for doc in docs], bins=np.arange(30))

In [ ]:
for ent in doc.ents:
    print(ent.label_)

In [ ]:
# annotator
# Emilia     688
# Matt      3032
# Rafael    1173
# Sam        166
# Sara      2963

## Read raw annotations

In [ ]:
annotations_raw = pd.DataFrame(tag_list)
annotations_raw.shape

In [ ]:
annotations_raw.note_type.value_counts()

In [ ]:
# annotations_raw.groupby()

In [ ]:
# pd.crosstab(annotations_raw.label, annotations_raw.note_type,)
# #.divide(annotations_raw.note_type.value_counts(),1)

#### parse `note_id` and expert name

In [ ]:
annotations = pd.concat([annotations_raw, 
           annotations_raw.filename.apply(lambda fn: pd.Series(parse_filename(fn)))],1)
annotations["annotator"] = annotations.annotator.str.title().map(lambda x: {"Raf": "Rafael",
                                                                           "Mat": "Matt"}.get(x,x))
annotations = annotations.sort_values(["filename", "start"])
annotations.shape

In [ ]:
for note_id, vv in annotations.groupby("note_id"):
    break

In [ ]:
vv.groupby("start")["end"].agg(lambda x:x.value_counts().index[0])

In [ ]:
vv.sort_values("start")

In [ ]:
# vv.sort_values("start")["span"]

In [ ]:
# from functools import reduce
# reduce( lambda x,y: x.overlaps(y), vv.sort_values("start")["span"].tolist())


In [ ]:
today_iso = date.today().isoformat()
outdir = f"results/pt-sample-2020-02-12-analysis-{today_iso}"

In [ ]:
os.makedirs(outdir, exist_ok=True)

In [ ]:
out_fn = f"{outdir}/annotations.csv"
print(out_fn)
annotations.to_csv(out_fn, index=False)

In [ ]:
# annotations[annotations.note_id.isnull()]